<a href="https://colab.research.google.com/github/SocTrep/Investigations/blob/main/GuyanaGoldMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#did

In [ ]:
#did it work this time?

In [ ]:
#save